In [4]:
import cv2
import time

In [8]:
def image_dealwith(img):
    #图像灰度处理
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)

    gradX = cv2.Sobel(gray, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)
    gradY = cv2.Sobel(gray, ddepth=cv2.CV_32F, dx=0, dy=1, ksize=-1)
    # subtract the y-gradient from the x-gradient
    gradient = cv2.subtract(gradX, gradY)
    gradient = cv2.convertScaleAbs(gradient)

    # blur and threshold the image
    blurred = cv2.blur(gradient, (9, 9))
    (_, thresh) = cv2.threshold(blurred, 45, 255, cv2.THRESH_BINARY)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 25))
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    # perform a series of erosions and dilations
    closed = cv2.erode(closed, None, iterations=4)
    closed = cv2.dilate(closed, None, iterations=4)
    return closed

In [ ]:
# camera = cv2.VideoCapture(0)
camera = cv2.VideoCapture(0)
time.sleep(0.25)

firstFrame = None


# 遍历视频的每一帧
while True:
    # 获取当前帧并初始化occupied/unoccupied文本
    (grabbed, frame) = camera.read()
 
    # 如果不能抓取到一帧，说明我们到了视频的结尾
    if not grabbed:
        break
 
    # 调整该帧的大小，转换为灰阶图像并且对其进行高斯模
    closed = image_dealwith(frame)
    (_,cnts, _) = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for c in cnts:
        rect = cv2.contourArea(c)
        (x, y, w, h) = cv2.boundingRect(c)
        crop_imgae = frame[y:y+h,x:x+w]
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # draw the text and timestamp on the frame
        # 在当前帧上写文字以及时间戳
        cv2.putText(frame, "Room Status: {}".format('text'), (x, y),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    #     cv2.putText(frame, "Room Status: {}".format(result), (10, 20),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        #显示当前帧并记录用户是否按下按键
        cv2.imshow("Security Feed", frame)
        key = cv2.waitKey(1)

        # 如果q键被按下，跳出循环
        if key == ord("q"):
            break
        
 # 清理摄像机资源并关闭打开的窗口
camera.release()
cv2.destroyAllWindows()